### Imports

In [ ]:
import sys
import os, shutil
import gc
import torch
import copy
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import open3d
import transforms3d
import open3d as o3d
from sklearn.neighbors import NearestNeighbors
import glob
from torch.utils.data import Dataset
from open3d import read_point_cloud
from tqdm import tqdm_notebook
import pandas as pd
from multiprocessing import Pool, cpu_count

In [ ]:
%matplotlib notebook

In [ ]:
def getint(name):
    return int(name.split('.')[0])

### Setting paths and constants

In [ ]:
# VOXEL_SZ = 0.2
# Z_MIN = 0
# INVERT_Z = False
# USE_GT = False
# EVERY_NTH_COUNT = 10
# PCD_TYPE = ".pcd"
# pcd_folder = "complete_extracted"

# VOXEL_SZ = 0.2
# Z_MIN = -2
# Z_MIN = 0
# INVERT_Z = True
# USE_GT = True
# EVERY_NTH_COUNT = 1
# PCD_TYPE = ".ply"
pcd_folder = "_out"

In [ ]:
# pcd_folder_path = "/home/sabyasachi/Projects/ati/data/data/datasets/IISC/2019-06-12/10-00-14-P1-6-auto-ccw_5loops_0.6_no_numba/"
pcd_folder_path = "/home/sabyasachi/Projects/ati/data/data/datasets/Carla/small_map/110k/static/1"

In [ ]:
pcd_path = os.path.join(pcd_folder_path, pcd_folder)

pcd_files = sorted(os.listdir(pcd_path), key=getint)
# FIRST_PCD = 100
# FIRST_PCD = int(pcd_files[0][:-4])
# FINAL_PCD = int(pcd_files[-1][:-4])

# OUTPUT_DIR = "./temp_save_parallel_dataset"

In [ ]:
VOXEL_SZ = 0.2

In [ ]:
idx = np.random.choice(len(pcd_files))
idx = 2925
prev_pcd_file = pcd_files[idx]
next_pcd_file = pcd_files[idx+10]

In [ ]:
idx

### My I/O fn

In [ ]:
def filter_pcd(old_pcd,
               apply_downsample = False,
               downsample_voxel_size = 0.2,
               
               apply_outlier_removal = False,
               downsample_radius = 1,
               downsample_neighbors = 20,
               
               apply_crop = True,
               crop_min_arr = np.array([-100,-100,-100]),
               crop_max_arr = np.array([100,100,2]),
               
               apply_cluster = False,
               cluster_neighbours = 30,
               cluster_labels = 2):
    np.random.seed(0)
    pcd = copy.deepcopy(old_pcd)
    
    if apply_outlier_removal:
        denser_pcd, ind = o3d.geometry.radius_outlier_removal(pcd,
                                                              nb_points = downsample_neighbors,
                                                              radius    = downsample_radius)
        pcd = denser_pcd
    
    if apply_downsample:
        voxel_down_pcd = o3d.geometry.voxel_down_sample(pcd, voxel_size = downsample_voxel_size)
        pcd = voxel_down_pcd
    
    if apply_crop:
        cropped_pcd = o3d.geometry.crop_point_cloud(pcd, crop_min_arr, crop_max_arr)
        pcd = cropped_pcd

    if apply_cluster:
        few_pts = np.asarray(pcd.points)
        try:
            few_pts_reduced = LocallyLinearEmbedding(n_neighbors=cluster_neighbours, n_components=2).fit_transform(few_pts)
        except Exception as e:
            try:
                few_pts_reduced = LocallyLinearEmbedding(n_neighbors=cluster_neighbours, n_components=2, eigen_solver='dense').fit_transform(few_pts)
            except Exception as e:
                few_pts_reduced = few_pts
        clf = MeanShift().fit(few_pts_reduced)
        pcd.points = o3d.utility.Vector3dVector(few_pts[clf.labels_ < cluster_labels])
    
    return pcd

def make_2d(pcd):
    new_pcd = copy.deepcopy(pcd)
    new_pts = np.concatenate([np.asarray(pcd.points)[:,:-1],np.zeros((len(pcd.points),1))], axis=1)
    new_pcd.points = o3d.utility.Vector3dVector(new_pts)
    return new_pcd

def read_pcd(pcd_id):
#     prefix = "".join(["0" for _ in range(3 - len(str(pcd_id)))])
    pcd_file = str(pcd_id) + PCD_TYPE
    pcd = o3d.io.read_point_cloud(os.path.join(pcd_path, pcd_file))
    return pcd

def draw_pcd(pcd, where='mat_2d'):    
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.show()
    elif where is 'mat_2d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], s=1)
        plt.show()
        
def pose2matrix(translation_list, rotation_angle_list):
    trans_vec = np.array(translation_list)
    rot_ang = [np.deg2rad(ang) for ang in rotation_angle_list ]
    rot_mat = transforms3d.euler.euler2mat(rot_ang[0], rot_ang[1], rot_ang[2])
    zoom = np.ones(3)
    transform_mat = transforms3d.affines.compose(trans_vec, rot_mat, zoom)
    return transform_mat

def draw_registration_result_2d(src_pcd, dst_pcd, x_pt, y_pt, theta):    
    src_pcd_tmp = copy.deepcopy(src_pcd)
    dst_pcd_tmp = copy.deepcopy(dst_pcd)
    
    src_pcd_tmp.paint_uniform_color([1, 0, 0])  # red source
    dst_pcd_tmp.paint_uniform_color([0, 0, 1])  # blue target
    
    transform_mat = pose2matrix([x_pt, y_pt, 0], [0,0,theta])
    dst_pcd_tmp.transform(transform_mat)
    
    visualizer = o3d.JVisualizer()
    visualizer.add_geometry(src_pcd_tmp)
    visualizer.add_geometry(dst_pcd_tmp)
    visualizer.show()
    
def draw_registration_result_3d(src, dst, trans_arr, rot_arr):
    source = copy.deepcopy(src)
    target = copy.deepcopy(dst)
    
    transform_mat = pose2matrix(trans_arr, rot_arr)
    
    source.paint_uniform_color([1, 0, 0]) # red
    target.paint_uniform_color([0, 0, 1]) # blue
    source.transform(transform_mat)
    
    visualizer = o3d.JVisualizer()
    visualizer.add_geometry(source)
    visualizer.add_geometry(target)
    visualizer.show()
    
def draw_registration_result(src, dst, transformation):
    source = copy.deepcopy(src)
    target = copy.deepcopy(dst)
    
    source.paint_uniform_color([1, 0, 0]) # red
    target.paint_uniform_color([0, 0, 1]) # blue
    source.transform(transformation)
#     o3d.visualization.draw_geometries([source_temp, target_temp], width=1280, height=800)
    visualizer = o3d.JVisualizer()
    visualizer.add_geometry(source)
    visualizer.add_geometry(target)
    visualizer.show()

In [ ]:
prev_pcd = o3d.io.read_point_cloud(os.path.join(pcd_path, prev_pcd_file))
next_pcd = o3d.io.read_point_cloud(os.path.join(pcd_path, next_pcd_file))

In [ ]:
def preprocess_point_cloud(pcd, voxel_size):
    pcd_down = o3d.geometry.voxel_down_sample(pcd, voxel_size)
    radius_normal = voxel_size * 2
    o3d.geometry.estimate_normals(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    radius_feature = voxel_size * 5
    pcd_fpfh = o3d.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

def prepare_dataset(source, target, voxel_size):
    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source_down, target_down, source_fpfh, target_fpfh

def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    result = o3d.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, distance_threshold,
        o3d.registration.TransformationEstimationPointToPoint(False), 4, [
            o3d.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.registration.RANSACConvergenceCriteria(4000000, 500))
    return result

def refine_registration(source, target, voxel_size, trans_init):
    distance_threshold = voxel_size * 0.4
    result = o3d.registration.registration_icp(
                source, target, distance_threshold, trans_init,
                o3d.registration.TransformationEstimationPointToPlane())
    return result

def get_pose(src, dst):
    
    source = filter_pcd(src)
    target = filter_pcd(dst)

    voxel_size = VOXEL_SZ
    source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(source, target, voxel_size)

    result_ransac = execute_global_registration(source_down, target_down,
                                                source_fpfh, target_fpfh,
                                                voxel_size)

    result_icp = refine_registration(source, target, voxel_size, result_ransac.transformation)
    
    transform_mat = result_icp.transformation
    trans_arr, rot_mat, scale_mat, shear_mat = transforms3d.affines.decompose44(transform_mat)
    rot_list = transforms3d.euler.mat2euler(rot_mat, axes='sxyz')
    rot_arr = np.array(rot_list)
    
    evaluation = o3d.registration.evaluate_registration(source, target, voxel_size * 5, pose2matrix(trans_arr, rot_arr))
    return trans_arr, rot_arr, evaluation.inlier_rmse

In [ ]:
trans_arr, rot_arr, rmse = get_pose(prev_pcd, next_pcd)

In [ ]:
rmse

In [ ]:
draw_registration_result(prev_pcd, next_pcd, pose2matrix(trans_arr, rot_arr))